# **Limpieza** Inmuebles24

#### LIBRERÍAS

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata # Función para eliminar acentos
from datetime import datetime, date  # Para trabajar con fechas y horas
from currency_converter import CurrencyConverter # convertidor de monedas
from dateutil.relativedelta import relativedelta  # Para calcular diferencias entre fechas, incluyendo años, meses y días

ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

#### CARGAR ARCHIVO

Solo cambia el nombre en 'archivo_filename' y su ruta con 'archivo_dir'

In [32]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
archivo_filename = "ruta/relativa/al/archivo.csv"
archivo_dir = os.path.join(directorio_actual)
archivo_path = os.path.join(archivo_dir, archivo_filename)

In [33]:
df = pd.read_csv('C:/xampp/htdocs/datalpine/resources/db/Scrapining/portales/Inmuebles_TulancingodeBravo_Septiembre2024.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   propiedad              47 non-null     object 
 1   tipo                   47 non-null     object 
 2   precio                 47 non-null     int64  
 3   denominacion           47 non-null     object 
 4   visualizaciones        43 non-null     float64
 5   metros_total           47 non-null     int64  
 6   metros_construido      47 non-null     int64  
 7   banos                  47 non-null     int64  
 8   medio_banos            47 non-null     int64  
 9   estacionamientos       47 non-null     int64  
 10  recamaras              47 non-null     int64  
 11  antiguedad             47 non-null     int64  
 12  tiempo_de_publicacion  47 non-null     object 
 13  status_desarrollo      47 non-null     object 
 14  ubicacion              47 non-null     object 
 15  url     

Elige el tipo de archivo a leer

In [ ]:
#df = pd.read_excel(archivo_path) 
df = pd.read_csv(archivo_path)

df.info()

#### LIMPIEZA NULOS

In [35]:
df.fillna({'precio': 0}, inplace=True) 
df.fillna({'denominacion': ''}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
df.fillna({'tiempo_de_publicacion': ''}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 
df.fillna({'visualizaciones': 0}, inplace=True) 
df.fillna({'antiguedad': 0}, inplace=True) 
df.fillna({'status_desarrollo': ''}, inplace=True) 
df.fillna({'latitud': 0}, inplace=True) 
df.fillna({'longitud': 0.}, inplace=True) 

# 1. Duplicados
* Estandarizar descripción
* Eliminación de los duplicados, dejando uno de los resultantes

In [36]:
df.columns

Index(['propiedad', 'tipo', 'precio', 'denominacion', 'visualizaciones',
       'metros_total', 'metros_construido', 'banos', 'medio_banos',
       'estacionamientos', 'recamaras', 'antiguedad', 'tiempo_de_publicacion',
       'status_desarrollo', 'ubicacion', 'url', 'descripcion', 'latitud',
       'longitud'],
      dtype='object')

In [37]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['descripcion'] = df['descripcion'].astype(str)
# Convertir a minúsculas
df['descripcion'] = df['descripcion'].str.lower()
# Eliminar acentos
def eliminar_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['descripcion'] = df['descripcion'].apply(eliminar_acentos)
df['descripcion'] = df['descripcion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

## a) Con todos los datos iguales

In [38]:
duplicados = df.loc[df.duplicated()]
print(duplicados)

                                            propiedad  tipo   precio  \
30   Casa Nueva Equipada en Fracc Privado, Tulancingo  Casa  1695000   
31  Casa en Venta en Fraccionamiento Pedregal de S...  Casa  7700000   
32  Casa con 3 Recámaras y 121 m de Terreno en Fra...  Casa  1995000   
33  Casa en Privada Arboledas, Col Rancho Nuevo, T...  Casa  1695000   
34  Casa en Fraccionamiento Cerrado, Cima Dorada T...  Casa   950000   
35  Casa Nueva en Privada San Daniel, Tulancingo H...  Casa  1150000   
37  Casa en Privada, Tulancingo de Bravo, Residenc...  Casa  3000000   

   denominacion  visualizaciones  metros_total  metros_construido  banos  \
30          MXN              0.0           105                120      2   
31          MXN              0.0           343                405      2   
32          MXN             67.0           121                175      3   
33          MXN             27.0           105                114      2   
34          MXN             43.0           

In [39]:
#df = df.drop_duplicates()
df = df.drop_duplicates(keep='first')

In [40]:
duplicados = df.loc[df.duplicated()]
print(duplicados)

Empty DataFrame
Columns: [propiedad, tipo, precio, denominacion, visualizaciones, metros_total, metros_construido, banos, medio_banos, estacionamientos, recamaras, antiguedad, tiempo_de_publicacion, status_desarrollo, ubicacion, url, descripcion, latitud, longitud]
Index: []


## b) Descripción y precio iguales

In [41]:
duplicados = df.loc[df.duplicated(subset=['descripcion','precio'])]
print(duplicados)

Empty DataFrame
Columns: [propiedad, tipo, precio, denominacion, visualizaciones, metros_total, metros_construido, banos, medio_banos, estacionamientos, recamaras, antiguedad, tiempo_de_publicacion, status_desarrollo, ubicacion, url, descripcion, latitud, longitud]
Index: []


In [42]:
df = df.drop_duplicates(subset=['descripcion','precio'],keep='first')

In [43]:
duplicados = df.loc[df.duplicated(subset=['descripcion','precio'])]
print(duplicados)

Empty DataFrame
Columns: [propiedad, tipo, precio, denominacion, visualizaciones, metros_total, metros_construido, banos, medio_banos, estacionamientos, recamaras, antiguedad, tiempo_de_publicacion, status_desarrollo, ubicacion, url, descripcion, latitud, longitud]
Index: []


## c) M2 construcción, Ubicación y Precio

In [44]:
duplicados = df.loc[df.duplicated(subset=['metros_construido','ubicacion','precio'])]
print(duplicados)

Empty DataFrame
Columns: [propiedad, tipo, precio, denominacion, visualizaciones, metros_total, metros_construido, banos, medio_banos, estacionamientos, recamaras, antiguedad, tiempo_de_publicacion, status_desarrollo, ubicacion, url, descripcion, latitud, longitud]
Index: []


In [45]:
df = df.drop_duplicates(subset=['metros_construido','ubicacion','precio'],keep='first')

In [46]:
duplicados = df.loc[df.duplicated(subset=['metros_construido','ubicacion','precio'])]
print(duplicados)

Empty DataFrame
Columns: [propiedad, tipo, precio, denominacion, visualizaciones, metros_total, metros_construido, banos, medio_banos, estacionamientos, recamaras, antiguedad, tiempo_de_publicacion, status_desarrollo, ubicacion, url, descripcion, latitud, longitud]
Index: []


## d) Características y Precio

In [47]:
duplicados = df.loc[df.duplicated(subset=['estacionamientos','recamaras','banos','medio_banos','precio'])]
print(duplicados)

                                            propiedad  tipo   precio  \
24   Casa Nueva Equipada en Fracc Privado, Tulancingo  Casa  1695000   
25  Casa en Privada Arboledas, Col Rancho Nuevo, T...  Casa  1695000   

   denominacion  visualizaciones  metros_total  metros_construido  banos  \
24          MXN              0.0           105                120      2   
25          MXN             27.0           105                114      2   

    medio_banos  estacionamientos  recamaras  antiguedad  \
24            0                 0          3           3   
25            0                 0          3           1   

      tiempo_de_publicacion status_desarrollo  \
24     Publicado desde ayer             venta   
25  Publicado hace 268 días             venta   

                          ubicacion  \
24  ,  Tulancingo de Bravo, Hidalgo   
25  ,  Tulancingo de Bravo, Hidalgo   

                                                  url  \
24  https://www.inmuebles24.com/propiedades/clasif

In [48]:
df = df.drop_duplicates(subset=['estacionamientos','recamaras','banos','medio_banos','precio'],keep='first')

In [49]:
duplicados = df.loc[df.duplicated(subset=['estacionamientos','recamaras','banos','medio_banos','precio'])]
print(duplicados)

Empty DataFrame
Columns: [propiedad, tipo, precio, denominacion, visualizaciones, metros_total, metros_construido, banos, medio_banos, estacionamientos, recamaras, antiguedad, tiempo_de_publicacion, status_desarrollo, ubicacion, url, descripcion, latitud, longitud]
Index: []


# 2. Manipulación de precios

## a) Precio en descripción

Agrega las diferentes formas de que el precio se encuentre en la descripción.

In [64]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\b(\d+[\.,]?\d*[\.,]?\d*)\s*($|mxn|USD|precio|pesos|monto|)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '')#.replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)

### Precio [0 / 0]

In [65]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


Eliminar registros que no recuperaron su precio ni desde la descripción

In [66]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

### Precio [0 / #]

In [67]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Precio [# / #]

In [68]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


## b) Denominación MXN

SI EXISTE UNA DENOMINACIÓN QUE NO SEA MXN, SE CONVIERTA EL PRECIO

In [ ]:
import datetime
c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'))
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
df['fecha_conversion'] = fecha_conversion
print(df[['precio', 'denominacion', 'precio_mxn', 'precio_usd', 'fecha_conversion']].head())

    precio denominacion  precio_mxn  precio_usd fecha_conversion
0  8000000          MXN     8000000   444999.64       2024-09-19
1  4810000          MXN     4810000   267556.03       2024-09-19
2  4500000          MXN     4500000   250312.30       2024-09-19
3  4300000          MXN     4300000   239187.31       2024-09-19
4  3946465          MXN     3946465   219521.94       2024-09-19


## c) Precio fuera del estandar

IDENTIFICAR Y ELIMINAR PROPIEDADES QUE CUENTEN CON PRECIOS BAJOS y ALTOS AL ESTANDAR

In [ ]:
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
df['precio'].sort_values()

1605      415000
1604      419710
1603      419770
1598      419770
1602      419770
          ...   
4       24000000
3       24000000
2       26000000
1       27000000
0       32000000
Name: precio, Length: 1607, dtype: int64

## D) Segmento o categoría

In [ ]:
# Crear un diccionario donde las llaves son las categorías y los valores son las tuplas de rangos
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

# Asignar la categoría a cada registro
df['categoria'] = df['precio'].apply(asignar_categoria)
print(df['categoria'].unique())

[None 'S1' 'A3' 'A2' 'A1' 'B3' 'B2' 'B1' 'C3' 'C2' 'C1' 'D3' 'D2' 'D1'
 'E3' 'E2' 'E1']


# 3. Meses transcurridos

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual

In [50]:
print(df['tiempo_de_publicacion'].unique())

['Publicado hace más de 1 año' 'Publicado hace 289 días'
 'Publicado hace 314 días' 'Publicado hace 67 días'
 'Publicado hace 252 días' 'Publicado hace 104 días'
 'Publicado hace 285 días' 'Publicado hace 48 días' 'Publicado desde ayer'
 'Publicado hace 18 días' 'Publicado hace 25 días'
 'Publicado hace 220 días' 'Publicado hace 55 días'
 'Publicado hace 216 días' 'Publicado hace 9 días'
 'Publicado hace 208 días' 'Publicado hace 37 días'
 'Publicado hace 64 días' 'Publicado hace 254 días'
 'Publicado hace 51 días' 'Publicado hace 44 días'
 'Publicado hace 163 días' 'Publicado hace 53 días'
 'Publicado hace 47 días' 'Publicado hace 84 días'
 'Publicado hace 63 días']


In [51]:
import numpy as np

# Extraer número de días
df['meses_transcurridos'] = df['tiempo_de_publicacion'].str.extract(r'(\d+)')
# Para manejar casos especiales como "Publicado desde ayer" o "más de 1 año"
df['meses_transcurridos'] = np.where(df['tiempo_de_publicacion'].str.contains('más de 1 año'), 365, df['meses_transcurridos'])
df['meses_transcurridos'] = np.where(df['tiempo_de_publicacion'].str.contains('ayer'), 1, df['meses_transcurridos'])
# Convertir a enteros para manipulación numérica
df['meses_transcurridos'] = df['meses_transcurridos'].astype(int)
df['meses_transcurridos'].unique()

array([365, 289, 314,  67, 252, 104, 285,  48,   1,  18,  25, 220,  55,
       216,   9, 208,  37,  64, 254,  51,  44, 163,  53,  47,  84,  63])

# 4. visualizaciones

VARIABLE QUEDA INTACTA

In [52]:
df['visualizaciones'].unique()

array([ 74., 119.,  66.,  39.,  91.,  75.,  67.,  62.,  38.,  44.,  51.,
        49.,  43.,  36.,  56.,   0.,  16.,  19.,  30.,  28.,  26.,  25.,
         9.,  65.,  22.,  47.,  41.,  32.])

# 5. AntigÜedad 

SE DEJA INTACTA POR EL MOMENTO

In [53]:
df['antiguedad'].unique()

array([   1,    8, 2019,    2,    0,    5,    3, 2022,   10,    9,   17,
          4,   30,   20,   18], dtype=int64)

# 6. Estado de desarrollo

In [54]:
df = df.rename(columns={'status_desarrollo': 'Status', })
df['Status'] .unique()

array(['venta'], dtype=object)

In [55]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['Status'] = df['Status'].astype(str)
# Convertir a minúsculas
df['Status'] = df['Status'].str.lower()
# Convertir la primera letra de cada palabra a mayúscula
df['Status'] = df['Status'].str.title()
df['Status'].unique()

array(['Venta'], dtype=object)

# 7. Tipo de propiedad

## a) Estandarizar

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['tipo'] = df['tipo'].astype(str)
# Convertir a minúsculas
df['tipo'] = df['tipo'].str.lower()
# Normaliza y elimina los acentos
def eliminar_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['tipo'] = df['tipo'].apply(eliminar_acentos)
# Convertir la primera letra de cada palabra a mayúscula
df['tipo'] = df['tipo'].str.title()
df['tipo'].unique()

array(['Casa', 'Casa En Fraccionamiento', 'Departamento',
       'Casa En Condominio', 'Condominio Horizontal'], dtype=object)

## b) Lista de tipos deseados

In [ ]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa En Condominio|Departamento|Departamento Compartido|Desarrollo Horizontal|Desarrollo Horizontal/Vertical|Desarrollo Vertical|Dúplex|'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

array(['Departamento', 'Condominio Horizontal', 'Casa', 'Dúplex',
       'Casa Dúplex', 'Casa en Fraccionamiento', 'Casa en Condominio'],
      dtype=object)

* Casas: Casa en condominio, 
* Departamento: Penthouse, Loft

## C) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

In [ ]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lote', case=False)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])#'descripcion',
    
    print("\n¿Te gustaría eliminar el registro?")
    # Solicitar la opción al usuario
    option = input("Elige una opción (si / no): ").strip().lower()
    
    if option == 'si':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

# 8. Metros terreno

## a) Metros en descripción

In [56]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales|mt2)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)

### Metros [0 / 0]

In [57]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [0 / #]

In [58]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [# / #]

In [59]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(f"metros_total: {row['metros_total']}")
    print(f"totales_desc: {row['totales_desc']}")
    print(f"url: {row['url']}")
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


## b) Manejo por rangos de UMA

Paso opcional por si no se espera cambiar el precio por el que se tenga en descripción

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

### DF cuando ambas columnas son 0 o vacíos

In [ ]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
totales = df.apply(actualizar_metros_total, axis=1)
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

#df = df.apply(actualizar_metros_total, axis=1)
print(totales[['metros_total', 'totales_desc', 'url']])

In [ ]:
# Si se aceptan los cambios:
df = df.apply(actualizar_metros_total, axis=1)

# 9. M2 construidos

## a) M-construidos en descripción

In [60]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)

### Metros [0 / 0]

In [61]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [0 / #]

In [62]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [# / #]

In [63]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 42:
metros_construido                                                                                                    460
construidos_desc                                                                                                   480.0
url                  https://www.inmuebles24.com/propiedades/clasificado/veclcain-casa-en-privada-cerrada-143278025.html
Name: 42, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'
3. Pasar al siguiente registro
Proceso completado.


# 10. Precio / m2

## M2 Totales (terreno)

In [ ]:
# Calcular precio por metro cuadrado del terreno
df['precio_m2_terreno'] = df['precio']/df['metros_total']

## M2 Construidos

In [ ]:
# Calcular precio por metro cuadrado de la cosntrucción
df['precio_m2_construido'] = df['precio']/df['metros_construido']

# 11. Estacionamientos

## a) Estacionamientos descritos

Asigna las diferentes maneras en las que puede encontrarse descrito el espacio de estacionamientos

In [ ]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)

### Estacionamientos [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)    
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Estacionamientos [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Estacionamientos [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [ ]:
df = df[df['estacionamientos'] <= 8]
#df[['estacionamientos','estacionamiento_desc','url']]

# 12. Recámaras

## a) Recámaras en descripción

In [ ]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)

### Recámaras [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Recámaras [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Mantener rango de recámaras <= 7 

In [ ]:
print(df['recamaras'].unique())
df = df[(df['recamaras'] <= 7)] 
#print(df[['recamaras','recamaras_desc','url']])

# 13. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [ ]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con medio baño en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])    
    print(row[['url']])
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

### Medio baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con valor en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_baños'] == 0) & (df['medios_baños'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_baños']==0) & (df['medios_baños'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_baños']!=0) & (df['medios_baños'].notna())) & (df['medio_baños'] != df['medios_baños']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

## B) Total de baños

In [ ]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_banos'] / 2)
df['bano_total'] = df['bano_total'].round(1)  # Round the total, not just 'banos'
print(df[['banos', 'medio_banos', 'bano_total']].head())

   banos  medio_banos  bano_total
0      5            0         5.0
1      0            0         0.0
2      0            0         0.0
3      1            0         1.0
4      1            0         1.0


# 14. Ubicación

## a) Desde descripción
#### ** Este paso aún necesita actualización en función de asignar los Códigos postales **

In [ ]:
# Función para extraer ubicación de la descripción
def extraer_ubicacion(descripcion):
    descripcion = descripcion.lower()
    # Patrones para buscar ubicaciones comunes
    patrones = [
        r'(?:ubicad[oa] en|en) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:calle|avenida|av\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:colonia|col\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:fraccionamiento|fracc\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})'
    ]
    2.
    for patron in patrones:
        match = re.search(patron, descripcion)
        if match:
            return match.group(1).strip().title()
    
    return None

# Aplicar la función al DataFrame
df['ubicacion_desc'] = df['descripcion'].apply(extraer_ubicacion)

#### Cuando la ubicación dice centro:

In [ ]:
# Filtrar los registros donde la ubicación contiene la palabra "centro"
df_centro = df[df['ubicacion'].str.contains('centro|Centro', case=False, na=False)]
num_registros_centro = len(df_centro)

print(f"Número de registros que contienen 'centro' en la ubicación: {num_registros_centro}")

Número de registros que contienen 'centro' en la ubicación: 823


In [ ]:

pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in df_centro.iterrows():
    print(f"Registro {idx}:")
    print(row[['ubicacion', 'ubicacion_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'ubicacion': ")
        df.at[idx, 'ubicacion'] = new_value  
    elif option == '2':
        # Sustituir el valor de 'ubicacion' por 'ubicacion_desc'
        df.at[idx, 'ubicacion'] = df.at[idx, 'ubicacion_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

## b) Separación

Se divide la ubicación por 
* Colonia
* Municipio
* Estado

In [ ]:
# Siempre regresar una lista de 3 elementos
def split_location(location):
    parts = location.split(',')
    parts += [None] * (3 - len(parts))
    return parts[:3] 
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].apply(split_location).apply(pd.Series)

# Limpiando valores nulos
df['Colonia'] = df['Colonia'].str.strip().fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].str.strip().fillna('Sin_municipio')
df['Estado'] = df['Estado'].str.strip().fillna('Sin_estado')
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]

#### Separación de tipo de lugar del nombre de la colonia

In [ ]:
# Añade las diferentes formas de identificar el tipo de lugar (etiquetas de ubicación)
tipos = ['Condominio', 'Fraccionamiento', 'Residencial', 'Colonia']
def obtener_tipo_lugar(nombre):
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return None

df['tipo_lugar'] = df['Colonia'].apply(obtener_tipo_lugar)
# Eliminar el tipo de lugar de la columna 'colonia'
df['Colonia'] = df.apply(lambda row: row['Colonia'].replace(row['tipo_lugar'], '').strip() if row['tipo_lugar'] else row['Colonia'], axis=1)
print(df[['Colonia','tipo_lugar','ubicacion']])

## c) Estandarizar valores 

#### Preparar que se deba tener el dato de 'Municipio', 'Estado'
Asegurate de asignar el Estado o Municipio que se esperaban dentro de las variables correspondientes

In [ ]:
Estado = 'Nombre de Estado esperado'
Municipio = 'Nombre de Municipio esperado'

df['Estado'] = df['Estado'].replace({'Sin_estado': Estado}, inplace=True)
df['Municipio'] = df['Municipio'].replace({'Sin_municipio': Municipio}, inplace=True)

#### Este paso es opcional, por ahora no es necesario usarlo ya que se usaba para el CP, pero estos no se tienen aún.

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['Colonia'] = df['Colonia'].astype(str)
# Convertir a minúsculas
df['Colonia'] = df['Colonia'].str.lower()
    # Normaliza y elimina los acentos
def eliminar_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)

# Reemplazar Ñ
#df['propiedad'] = df['propiedad'].str.replace('ñ', 'n')
df['Colonia'] = df['Colonia'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# Convertir la primera letra de cada palabra a mayúscula
#df['Colonia'] = df['Colonia'].str.title()

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['Municipio'] = df['Municipio'].astype(str)
# Convertir a minúsculas
df['Municipio'] = df['Municipio'].str.lower()

def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['Municipio'] = df['Municipio'].apply(eliminar_acentos)

# Reemplazar Ñ
#df['propiedad'] = df['propiedad'].str.replace('ñ', 'n')
df['Municipio'] = df['Municipio'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# Convertir la primera letra de cada palabra a mayúscula
#df['Colonia'] = df['Colonia'].str.title()

## d) Mapeo de CP y tipo de lugar por diccionario

#### Separación de tipo de lugar del nombre de la colonia
**PASO QUE NO SE TIENE COMPLETO**

In [ ]:
# Cargar todas las hojas del archivo Excel en un diccionario de DataFrames
#ruta_excel = 'cp_dictionary_estandarizado.xlsx'
ruta_excel = "../cp_dictionary_estandarizado.xlsx"
hojas = pd.read_excel(ruta_excel, sheet_name=None)

# Obtener combinaciones únicas de Estado, Municipio y Colonia
combinaciones_unicas = df[['Estado', 'Municipio', 'Colonia']].drop_duplicates()

# Crear un diccionario para almacenar los códigos postales y tipos de asentamiento encontrados
codigos_postales = {}
tipos_asentamiento = {}

# Función para limpiar texto (minúsculas y sin acentos)
def limpiar_texto(texto):
    texto = texto.lower().strip()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    return texto

# Buscar el código postal y tipo de asentamiento para cada combinación única
for _, row in combinaciones_unicas.iterrows():
    estado = limpiar_texto(row['Estado'])
    municipio = limpiar_texto(row['Municipio'])
    colonia = limpiar_texto(row['Colonia'])

    # Verificar si el estado existe en las hojas del Excel
    if estado in hojas:
        df_estado = hojas[estado]

        # Filtrar por municipio y colonia
        df_filtrado = df_estado[
            (df_estado['D_mnpio'].apply(limpiar_texto) == municipio) & 
            (df_estado['d_asenta'].apply(limpiar_texto) == colonia)
        ]

        # Si hay coincidencias, almacenar el primer código postal y tipo de asentamiento en los diccionarios
        if not df_filtrado.empty:
            codigos_postales[(estado, municipio, colonia)] = df_filtrado['d_codigo'].iloc[0]
            tipos_asentamiento[(estado, municipio, colonia)] = df_filtrado['d_tipo_asenta'].iloc[0]
        else:
            codigos_postales[(estado, municipio, colonia)] = None
            tipos_asentamiento[(estado, municipio, colonia)] = None
    else:
        codigos_postales[(estado, municipio, colonia)] = None
        tipos_asentamiento[(estado, municipio, colonia)] = None

# Crear columnas 'CP' y 'Tipo_Asentamiento' en el DataFrame original usando los diccionarios
df['CP'] = df.apply(lambda x: codigos_postales.get(
    (limpiar_texto(x['Estado']), limpiar_texto(x['Municipio']), limpiar_texto(x['Colonia']))), axis=1)
df['Tipo_Asentamiento'] = df.apply(lambda x: tipos_asentamiento.get(
    (limpiar_texto(x['Estado']), limpiar_texto(x['Municipio']), limpiar_texto(x['Colonia']))), axis=1)

# Obtener las combinaciones únicas de las columnas 'Colonia', 'Municipio', 'Estado', 'CP', 'Tipo_Asentamiento'
combinaciones_unicas = df[['Colonia', 'Municipio', 'Estado', 'CP', 'Tipo_Asentamiento']].drop_duplicates()

# Mostrar resultados
for idx, row in combinaciones_unicas.iterrows():
    print(f"Combinación {idx + 1}:")
    print(f"Colonia: {row['Colonia']}, Municipio: {row['Municipio']}, Estado: {row['Estado']}, CP: {row['CP']}, Tipo_Asentamiento: {row['Tipo_Asentamiento']}")
    print("---")

# 15. Latitud y Longitud

In [ ]:
df['latitud','longitud']


____
# 11. Variables oficiales

In [ ]:
df = df.drop(['denominacion', 'precio_desc','baños_desc','baños_completos','medios_baños',], axis=1, errors='ignore') #'fecha_conversion'

In [ ]:
# Tiempo de publicación limpio:
df['tiempo_de_publicacion'] = df['fecha_estandarizada']

EXTRA

In [ ]:
df['id'] = range(1, len(df) + 1)

ORDEN OFICIAL

In [ ]:
columnas_ordenadas = ['id', 'categoria', 'precio','precio_mxn','precio_usd','fecha_conversion', 'propiedad', 'metros_total', 'metros_construido',
                      'precio_m2_terreno', 'precio_m2_construido','tiempo_de_publicacion', 'meses_transcurridos','meses_transcurridos_fecha', 'tipo','Status',
                      'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total','seguridad_privada', 'tipo_lugar','Colonia','Municipio','Estado','CP','ubicacion','url','descripcion']
df = df[columnas_ordenadas]

In [ ]:
df.info()

# 13. Exportar

Cambia el nombre por el que desees en 'archivo'

In [22]:
archivo = 'nombre_del_archivo'
ext = '.csv'
archivo_export = archivo + ext
df.to_csv(archivo_export, encoding='utf-8', index=False)